In [ ]:
!pip3 install kaggle
!pip3 install pandas

In [ ]:
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

# # Download the dataset as a ZIP file
api.dataset_download_files('kaggle/meta-kaggle')

In [ ]:
import zipfile
# # Unzip the dataset
with zipfile.ZipFile('meta-kaggle.zip', 'r') as zip_ref:
     zip_ref.extractall()

In [ ]:
import sqlite3

conn = sqlite3.connect('Kaggle.db')
c = conn.cursor()
# c.execute('PRAGMA foreign_keys=on;')
# conn2 = sqlite3.connect('Kaggle.db')
# c2 = conn2.cursor()

In [ ]:
tables = {
    'Users': ['Id INT PRIMARY KEY', 'UserName TEXT', 'DisplayName TEXT', 'RegisterDate DATE', 'PerformanceTier INT'],
    'Tags': ['Id INT PRIMARY KEY', 'ParentTagId INT', 'Name TEXT', 'Slug TEXT', 'FullPath TEXT', 'Description TEXT', 'DatasetCount INT', 'CompetitionCount INT', 'KernelCount INT'],
    'Organizations': ['Id INT PRIMARY KEY', 'Name TEXT', 'Slug TEXT', 'CreationDate DATE', 'Description TEXT'],
    'KernelLanguages': ['Id INT PRIMARY KEY', 'Name TEXT', 'DisplayName TEXT', 'IsNotebook BOOLEAN'],
    'Forums': ['Id INT PRIMARY KEY', 'ParentForumId INT', 'Title TEXT'],
    'Datasets': ['Id INT PRIMARY KEY', 'CreatorUserId INT', 'OwnerUserId INT', 'OwnerOrganizationId INT', 'CurrentDatasetVersionId INT', 'CurrentDatasourceVersionId INT', 'ForumId INT', 'Type TEXT', 'CreationDate DATE', 'LastActivityDate DATE','TotalViews INT','TotalDownloads INT','TotalVotes INT','TotalKernels INT', 'FOREIGN KEY(CreatorUserId) REFERENCES Users(Id)'],
    'DatasetVersions': ['Id INT PRIMARY KEY', 'DatasetId INT', 'DatasourceVersionId INT', 'CreatorUserId INT', 'LicenseName TEXT', 'CreationDate DATE', 'VersionNumber INT', 'Title TEXT', 'Slug TEXT', 'Subtitle TEXT', 'FOREIGN KEY(DatasetId) REFERENCES Datasets(Id)', 'FOREIGN KEY(CreatorUserId) REFERENCES Users(Id)'],
    'KernelVersions': ['Id INT PRIMARY KEY', 'ScriptId INT', 'ParentScriptVersionId INT', 'ScriptLanguageId INT', 'AuthorUserId INT', 'CreationDate DATE', 'VersionNumber INT', 'Title TEXT', 'EvaluationDate DATE', 'IsChange BOOLEAN','TotalLines INT','LinesInsertedFromPrevious INT','LinesChangedFromPrevious INT','LinesUnchangedFromPrevious INT','LinesInsertedFromFork INT','LinesDeletedFromFork INT','LinesChangedFromFork INT','LinesUnchangedFromFork INT','TotalVotes INT', 'FOREIGN KEY(AuthorUserId) REFERENCES Users(Id)', 'FOREIGN KEY(ScriptLanguageId) REFERENCES KernelLanguages(Id)'],
    'Kernels': ['Id INT PRIMARY KEY', 'AuthorUserId INT', 'CurrentKernelVersionId INT', 'ForkParentKernelVersionId INT', 'ForumTopicId INT', 'FirstKernelVersionId INT', 'CreationDate DATE', 'EvaluationDate DATE', 'MadePublicDate DATE', 'IsProjectLanguageTemplate BOOLEAN','TotalViews INT','TotalComments INT','TotalVotes INT', 'FOREIGN KEY(AuthorUserId) REFERENCES Users(Id)', 'FOREIGN KEY(CurrentKernelVersionId) REFERENCES KernelVersions(Id)'],
    'Competitions': ['Id INT PRIMARY KEY', 'Slug TEXT', 'Title TEXT', 'Subtitle TEXT', 'HostSegmentTitle TEXT', 'ForumId INT', 'OrganizationId INT', 'EnabledDate DATE', 'DeadlineDate DATE'],
    'Teams': ['Id INT PRIMARY KEY', 'CompetitionId INT', 'TeamLeaderId INT', 'TeamName TEXT', 'ScoreFirstSubmittedDate DATE', 'LastSubmissionDate DATE', 'PublicLeaderboardSubmissionId INT', 'PrivateLeaderboardSubmissionId INT', 'IsBenchmark BOOLEAN', 'Medal INT', 'PrivateLeaderboardRank INT','FOREIGN KEY(CompetitionId) REFERENCES Competitions(Id)', 'FOREIGN KEY(TeamLeaderId) REFERENCES Users(Id)'],
    'ForumTopics': ['Id INT PRIMARY KEY', 'ForumId INT', 'KernelId INT', 'LastForumMessageId INT', 'FirstForumMessageId INT', 'CreationDate DATE', 'LastCommentDate DATE', 'Title TEXT', 'IsSticky BOOLEAN', 'TotalViews INT','Score INT','TotalMessages INT','TotalReplies INT', 'FOREIGN KEY(ForumId) REFERENCES Forums(Id)'],
    'ForumMessages': ['Id INT PRIMARY KEY', 'ForumTopicId INT', 'PostUserId INT', 'PostDate DATE', 'ReplyToForumMessageId INT', 'Message TEXT', 'Medal INT', 'MedalAwardDate DATE', 'FOREIGN KEY(PostUserId) REFERENCES Users(Id)','FOREIGN KEY(ForumTopicId) REFERENCES ForumTopics(Id)'],
    'Submissions': ['Id INT PRIMARY KEY', 'SubmittedUserId INT', 'TeamId INT', 'SourceKernelVersionId INT', 'SubmissionDate DATE', 'ScoreDate DATE', 'IsAfterDeadline BOOLEAN', 'PublicScoreLeaderboardDisplay FLOAT', 'PublicScoreFullPrecision FLOAT', 'PrivateScoreLeaderboardDisplay FLOAT', 'FOREIGN KEY(SubmittedUserId) REFERENCES Users(Id)','FOREIGN KEY(TeamId) REFERENCES Teams(Id)'],
    'CompetitionTags': ['Id INT PRIMARY KEY', 'CompetitionId INT', 'TagId INT', 'FOREIGN KEY(CompetitionId) REFERENCES Competitions(Id)', 'FOREIGN KEY(TagId) REFERENCES Tags(Id)'],
    'DatasetTags': ['Id INT PRIMARY KEY', 'DatasetId INT', 'TagId INT', 'FOREIGN KEY(DatasetId) REFERENCES Datasets(Id)', 'FOREIGN KEY(TagId) REFERENCES Tags(Id)'],
    'DatasetTasks': ['Id INT PRIMARY KEY', 'DatasetId INT', 'OwnerUserId INT', 'CreationDate DATE', 'Description TEXT', 'ForumId INT', 'Title TEXT', 'Subtitle TEXT', 'Deadline DATE', 'TotalVotes INT', 'FOREIGN KEY(DatasetId) REFERENCES Datasets(Id)', 'FOREIGN KEY(OwnerUserId) REFERENCES Users(Id)'],
    'UserOrganizations': ['Id INT PRIMARY KEY', 'UserId INT', 'OrganizationId INT', 'JoinDate DATE', 'FOREIGN KEY(UserId) REFERENCES Users(Id)', 'FOREIGN KEY(OrganizationId) REFERENCES Organizations(Id)'],
    'Datasources': ['Id INT PRIMARY KEY', 'CreatorUserId INT', 'CreationDate DATE', 'Type TEXT', 'CurrentDatasourceVersionId INT', 'FOREIGN KEY(CreatorUserId) REFERENCES Users(Id)'],
    'DatasetTaskSubmissions': ['Id INT PRIMARY KEY', 'DatasetTaskId INT', 'SubmittedUserId INT', 'CreationDate DATE', 'KernelId INT', 'DatasetId INT', 'AcceptedDate DATE', 'FOREIGN KEY(DatasetTaskId) REFERENCES DatasetTasks(Id)', 'FOREIGN KEY(SubmittedUserId) REFERENCES Users(Id)', 'FOREIGN KEY(DatasetId) REFERENCES Datasets(Id)','FOREIGN KEY(KernelId) REFERENCES Kernels(Id)'],
    'KernelTags': ['Id INT PRIMARY KEY', 'KernelId INT', 'TagId INT', 'FOREIGN KEY(TagId) REFERENCES Tags(Id)','FOREIGN KEY(KernelId) REFERENCES Kernels(Id)'],
    'KernelVersionCompetitionSources': ['Id INT PRIMARY KEY', 'KernelVersionId INT', 'SourceCompetitionId INT', 'FOREIGN KEY(SourceCompetitionId) REFERENCES Competitions(Id)', 'FOREIGN KEY(KernelVersionId) REFERENCES KernelVersions(Id)'],
    'KernelVersionDatasetSources': ['Id INT PRIMARY KEY', 'KernelVersionId INT', 'SourceDatasetVersionId INT', 'FOREIGN KEY(SourceDatasetVersionId) REFERENCES DatasetVersions(Id)', 'FOREIGN KEY(KernelVersionId) REFERENCES KernelVersions(Id)'],
    'KernelVersionKernelSources': ['Id INT PRIMARY KEY', 'KernelVersionId INT', 'SourceKernelVersionId INT', 'FOREIGN KEY(KernelVersionId) REFERENCES KernelVersions(Id)'],
    'KernelVotes': ['Id INT PRIMARY KEY', 'UserId INT', 'KernelVersionId INT', 'VoteDate DATE', 'FOREIGN KEY(UserId) REFERENCES Users(Id)', 'FOREIGN KEY(KernelVersionId) REFERENCES KernelVersions(Id)'],
    'DatasetVotes': ['Id INT PRIMARY KEY', 'UserId INT', 'DatasetVersionId INT', 'VoteDate DATE', 'FOREIGN KEY(UserId) REFERENCES Users(Id)', 'FOREIGN KEY(DatasetVersionId) REFERENCES DatasetVersions(Id)'],
    'TeamMemberships': ['Id INT PRIMARY KEY', 'TeamId INT', 'UserId INT', 'RequestDate DATE', 'FOREIGN KEY(TeamId) REFERENCES Teams(Id)', 'FOREIGN KEY(UserId) REFERENCES Users(Id)'],
    'UserAchievements': ['Id INT PRIMARY KEY', 'UserId INT', 'AchievementType TEXT', 'Tier INT', 'TierAchievementDate DATE', 'Points INT', 'CurrentRanking INT', 'HighestRanking INT', 'TotalGold INT', 'TotalSilver INT','TotalBronze INT', 'FOREIGN KEY(UserId) REFERENCES Users(Id)'],
    'UserFollowers': ['Id INT PRIMARY KEY', 'UserId INT', 'FollowingUserId INT', 'CreationDate DATE', 'FOREIGN KEY(UserId) REFERENCES Users(Id)', 'FOREIGN KEY(FollowingUserId) REFERENCES Users(Id)'],
    'ForumMessageVotes': ['Id INT PRIMARY KEY', 'ForumMessageId INT', 'FromUserId INT', 'ToUserId INT', 'VoteDate DATE', 'FOREIGN KEY(FromUserId) REFERENCES Users(Id)', 'FOREIGN KEY(ToUserId) REFERENCES Users(Id)'],
    'Episodes': ['Id INT PRIMARY KEY', 'Type TEXT', 'CompetitionId INT', 'CreateTime DATE', 'EndTime DATE', 'FOREIGN KEY(CompetitionId) REFERENCES Competitions(Id)'],
    'EpisodeAgents': ['Id INT PRIMARY KEY', 'EpisodeId INT', 'Reward INT', 'State TEXT', 'SubmissionId INT', 'InitialConfidence FLOAT', 'InitialScore FLOAT', 'UpdatedConfidence FLOAT', 'UpdatedScore FLOAT', 'FOREIGN KEY(EpisodeId) REFERENCES Episodes(Id)', 'FOREIGN KEY(SubmissionId) REFERENCES Submissions(Id)']
}


In [ ]:
# Create all tables
for table_name, columns in tables.items():
    # First, check if the table exists
    c.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';")
    
    # If the table does not exist, create it
    if c.fetchone() is None:
        query = f"CREATE TABLE {table_name} ({', '.join(columns)});"
        c.execute(query)

In [ ]:
import pandas as pd

chunksize = 1000000

# Step 1: Insert only 10,000 users
# user_chunk = pd.read_csv('Users.csv', nrows=10000)
# user_chunk.to_sql('Users', conn, if_exists='replace', index=False)  

# Step 2: Insert other tables
# Exclude 'Users' as we have already inserted
for table_name, columns in tables.items():
    # if table_name == 'Users':
    #     continue

    print(f"Processing {table_name}")
    # column_names = [col.split()[0] for col in columns]
    # query = f'SELECT * FROM {table_name}'
    # data = pd.read_sql(query, conn2)
    # try:
    #     data.to_sql(table_name,conn, if_exists='replace', index=False)
    # except (sqlite3.IntegrityError, sqlite3.OperationalError) as e:
        # print(f"Skipping chunk due to error: {e}")
    for chunk in pd.read_csv(f"{table_name}.csv", chunksize=chunksize):
        print(f"Processing chunk for {table_name}")
        # matching_columns = [col for col in chunk.columns if col in column_names]
        # chunk = chunk[matching_columns]
        try:
            chunk.to_sql(table_name, conn, if_exists='append', index=False)
        except (sqlite3.IntegrityError, sqlite3.OperationalError) as e:
            print(f"Skipping chunk due to error: {e}")

In [ ]:
import sqlite3

conn = sqlite3.connect('Kaggle.db')
c = conn.cursor()

In [ ]:
import pandas as pd

data_df = pd.read_sql('''
SELECT 
    U.Id AS UserId,
    U.UserName,
    U.DisplayName,
    U.PerformanceTier,
    T.CompetitionId,
    C.Slug AS CompetitionSlug,
    C.Title AS CompetitionTitle,
    C.Subtitle AS CompetitionSubTitle,
    C.EnabledDate,
    T.Id AS TeamId,
    T.TeamName,
    T.Medal,
    T.PrivateLeaderboardRank,
    julianday('now') - julianday(substr(T.LastSubmissionDate, 7, 4) || '-' || substr(T.LastSubmissionDate, 1, 2) || '-' || substr(T.LastSubmissionDate, 4, 2)) AS DaysFromLastSubmission
FROM 
    Users U
JOIN 
    TeamMemberships TM ON U.Id = TM.UserId
JOIN 
    Teams T ON TM.TeamId = T.Id
JOIN 
    Competitions C ON T.CompetitionId = C.Id
WHERE 
    T.PrivateLeaderboardRank IS NOT NULL AND
    substr(C.EnabledDate, 7, 4) || '-' || substr(C.EnabledDate, 4, 2) || '-' || substr(C.EnabledDate, 1, 2) >= strftime('%Y-%m-%d', 'now', '-3 years')
''', conn)

In [ ]:
import numpy as np

# Get distinct CompetitionIds
distinct_competition_ids = data_df['CompetitionId'].unique()

# Calculate 20% of the number of distinct CompetitionIds
num_ids_to_select = int(len(distinct_competition_ids) * 0.2)

# Select 20% of the CompetitionIds randomly
selected_competition_ids = np.random.choice(distinct_competition_ids, size=num_ids_to_select, replace=False)

# Storing these IDs in a constant
CONST_SELECTED_COMPETITION_IDS = selected_competition_ids

# Printing the constant for confirmation
CONST_SELECTED_COMPETITION_IDS

# Convert the array of IDs to a string format suitable for SQL query
ids_to_exclude = ','.join(map(str, CONST_SELECTED_COMPETITION_IDS))

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('Kaggle.db')
c = conn.cursor()
ids_to_exclude = ('32093,21328,44631,23622,38321,20927,32281,21017,27747,29464,27929,27994,21408,32132,32392,32524,27474,31644,29335,29105,32811,34419,21147,25590,35628,25925,33501,29193,34034,20464,19991,20112,20315,32094,21038,35126,30971,34713,21467,34024,24323,34903,26940,23686,32465,31082,31256,35366,31492,21345,26937,31084,20274,31344,20139,25787,34786,35681,26819,31833,23558,26562,33630,23291,30572,20082,26454,20914,34068,31363,27467,20883,19010,21325,31991,18685,23675,19693,29106,24309,19231,26079,19702,20211,20949,28007,32176,24951,25521,19990,26630,27714,29041,32475,33580,32272,35420,39585,26343,34523,30639,32351,18045,22725,32400,45917,24494,32616,18647,27318,26518,31081,34683,29793,26622,35136,24859,29478,35385,26607,21906,26943,25698,19013,25303,34342,22359,32373,26633,35467,35638,31522,31738,26941,32288,34097,29343,19726,32170,25697,22450,29770,23324,33089,25875,27800,26861,32612,21810,34636,25191,26923,31405,33656,18237,34290,31817,32105,26452,32147,34524,35798,26775,22210,33400,23327,32727,32845,30862,31334,21572,26069,30919,26423,13836,32230,33349,30719,30241,33133,24726,25404,27169,27781,26178,35015,27907,31491,31085,35384,33049,33078,22154,20865,35245,20326,24281,26811,31458,19629,31881,22804,21867,34349,24373,29807,20533,34954,59109,30937,36010,18894,29663,20288,31843,23679,34198,23556,31185,21434,29104,29144,24327,25914,22422,19237,21849,29803,30833,26659,22597,34179,26663,27962,25842,34337,24891,20357,27941,33250,22554,20441,19062,33431,33506,30874,30605,22936,22760,33418,25951,33904,20161,33353,52279,31406,20230,32548,24741,31404,24334,26192,20212,26930,34166,27377,27040,20289,33106,34547,35745,31548,22962,29533,34600,21837,34407,19079,33177,34570,26875,20159,30425,18588,32118,35853,27783,35893,34574,33567,27000,32437,18905,30905,32838,30734,25808,25950,20951,31640,31518,22512,34308,30344,29056,20009,23401,22572,34226,24761,20096,31847,22738,34229,33119,35297,19052,25908,30591,32452,24754,27151,20331,26288,31274,24078,22123,32358,34017,30801,35743,31683,26611,31767,29347,27835,35004,33607,24732,22266,27644,27263,52732,35166,25530,35291,26597,22032,24230,20069,33244,34899,26211,27971,32657,29282,33039,23526,23486,18976,30907,29175,20667,29166,24161,18945,25458,29486,20219,44630,18605,26600,34930,27643,27219,24298,23721,18325,26578,29173,29908,34270,31219,33352,21347,30038,20043,21392,33004,30550,32599,29483,34886,27784,26470,33665,18191,19626,25341,25288,25297,24301,22091,21707,35511,35270,25513,18011,34686,32223,21341,22048,31320,35598,32977,21338,19772,30466,26464,29148,26688,25401,23550,31197,34834,33367,35367,26480,33105,35685,31624,27618,20845,35306,22873,20542,24233,25414,31155,24673,24397,32506,34542,24084,31418,29723,33647,22557,20866,33226,26421,34785,25625')

data_df = pd.read_sql(
f'''
SELECT 
    U.Id AS UserId,
    U.UserName,
    U.DisplayName,
    U.PerformanceTier,
    T.CompetitionId,
    C.Slug AS CompetitionSlug,
    C.Title AS CompetitionTitle,
    C.Subtitle AS CompetitionSubTitle,
    C.EnabledDate,
    T.Id AS TeamId,
    T.TeamName,
    T.Medal,
    T.PrivateLeaderboardRank,
    julianday('now') - julianday(substr(T.LastSubmissionDate, 7, 4) || '-' || substr(T.LastSubmissionDate, 1, 2) || '-' || substr(T.LastSubmissionDate, 4, 2)) AS DaysFromLastSubmission
FROM 
    Users U
JOIN 
    TeamMemberships TM ON U.Id = TM.UserId
JOIN 
    Teams T ON TM.TeamId = T.Id
JOIN 
    Competitions C ON T.CompetitionId = C.Id
WHERE 
    T.PrivateLeaderboardRank IS NOT NULL AND
    substr(C.EnabledDate, 7, 4) || '-' || substr(C.EnabledDate, 4, 2) || '-' || substr(C.EnabledDate, 1, 2) >= strftime('%Y-%m-%d', 'now', '-3 years') AND
    C.Id NOT IN ({ids_to_exclude})
'''
, conn)
users_df = data_df[['UserId', 'UserName', 'DisplayName', 'PerformanceTier']]
users_df = users_df.drop_duplicates(subset='UserId', keep='first')
data_df.head()

In [ ]:
tags_df = pd.read_sql(
    f'''
    SELECT
    UserId,
    TagId,
    TagName,
    COUNT(*) AS Weight
    FROM (
    SELECT
        u.Id AS UserId,
        t.id AS TagId,
        t.Name AS TagName,
        'Kernel' AS Source
    FROM
        Kernels k
    JOIN KernelTags kt ON k.Id = kt.KernelId
    JOIN Tags t ON kt.TagId = t.Id
    JOIN Users u ON k.AuthorUserId = u.Id
    UNION ALL
    SELECT
        u.Id AS UserId,
        t.id AS TagId,
        t.Name AS TagName,
        'Dataset' AS Source
    FROM
        Datasets ds
    JOIN DatasetTags dt ON ds.Id = dt.DatasetId
    JOIN Tags t ON dt.TagId = t.Id
    JOIN Users u ON ds.CreatorUserId = u.Id
    UNION ALL
    SELECT
        u.Id AS UserId,
        t.id AS TagId,
        t.Name AS TagName,
        'Competition' AS Source
    FROM
        Competitions c
    JOIN CompetitionTags ct ON c.Id = ct.CompetitionId
    JOIN Tags t ON ct.TagId = t.Id
    JOIN Teams tm ON c.Id = tm.CompetitionId
    JOIN TeamMemberships tm2 ON tm.Id = tm2.TeamId
    JOIN Users u ON tm2.UserId = u.Id
    WHERE ct.CompetitionId NOT IN ({ids_to_exclude})
    ) AS CombinedUserInfo
    GROUP BY UserId, TagId
    '''
    , conn)
tags_df = pd.merge(users_df['UserId'],tags_df, on="UserId", how="left")
tags_df['TagName'].fillna('No_Tags', inplace=True)
tags_df['Weight'].fillna(0, inplace=True)
tags_df['TagId'].fillna(0, inplace=True)
tags_df.head()
tags_df.to_sql('tags', conn2, if_exists='replace', index=False)

In [ ]:
conn2 = sqlite3.connect('data3.db')
c2 = conn2.cursor()
users_df.to_sql('users', conn2, if_exists='replace', index=False)

In [ ]:
competitions_df = data_df[['UserId', 'CompetitionId','CompetitionSlug','CompetitionTitle','CompetitionSubTitle','EnabledDate']].copy()
teams_df = data_df[['UserId','CompetitionId','TeamId','TeamName','DaysFromLastSubmission','PrivateLeaderboardRank','Medal']].copy()
teams_df['Medal'].fillna(4, inplace=True)
teams_df['Medal'].replace(0, 4, inplace=True)
competitions_df.to_sql('competitions', conn2, if_exists='replace', index=False)
teams_df.to_sql('teams', conn2, if_exists='replace', index=False)

In [ ]:
test_df = pd.read_sql(
f'''
SELECT 
    c.Id, 
    c.Title, 
    c.Subtitle, 
    c.Slug,
    GROUP_CONCAT(t.Name, ', ') AS Tags
FROM 
    Competitions c
LEFT JOIN 
    CompetitionTags ct ON ct.CompetitionId = c.Id
LEFT JOIN 
    Tags t ON t.Id = ct.TagId
WHERE 
    c.Id IN ({ids_to_exclude})
GROUP BY 
    c.Id
'''
, conn)
test_df.head()
test_df['Tags'].fillna('No_Tags', inplace=True)
test_df.to_sql('test_data', conn2, if_exists='replace', index=False)